In [1]:
!pip install opencv-python dlib


DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install opencv-python dlib imutils

DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
1. 카메라 행렬 설정
카메라 행렬(camera_matrix)의 값이 실제 카메라의 초점 거리와 센서 크기에 맞지 않으면 정확한 포즈 추정이 어렵습니다. 현재는 화면의 가로 크기를 초점 거리로 사용하고 있지만, 실제 카메라의 매개변수를 사용해야 합니다.

2. 왜곡 계수
왜곡 계수(dist_coeffs)를 0으로 설정했는데, 실제 카메라의 왜곡 계수를 고려해야 할 수 있습니다. 실제 왜곡 계수를 모르더라도, 0으로 설정하는 것은 기본값입니다.

3. 오일러 각도 추출
cv2.decomposeProjectionMatrix를 사용하여 오일러 각도를 추출하는 방법은 좀 복잡할 수 있습니다. 오일러 각도를 직접 계산하거나 다른 방법으로 포즈를 추정할 수 있습니다.

4. PnP 알고리즘의 정확성
cv2.solvePnP의 결과가 정확하지 않을 수 있습니다. 모델 포인트와 이미지 포인트의 매칭이 잘못되었을 수 있습니다.

아래는 개선된 코드로, 카메라 매트릭스와 왜곡 계수를 좀 더 정확하게 설정하고, 회전 벡터에서 직접 피치와 요 각도를 추정하는 방법을 사용합니다.

In [36]:
!pip install torch torchvision transformers


DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

def is_palm_facing_camera(hand_landmarks):
    """손바닥이 카메라를 향하고 있는지 확인하는 함수"""
    wrist = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x,
                      hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y,
                      hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].z])
    
    index_finger_tip = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                                 hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y,
                                 hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z])
    
    thumb_tip = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                          hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                          hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].z])
    
    # 손목과 손끝 사이의 벡터를 계산
    wrist_to_index = index_finger_tip - wrist
    wrist_to_thumb = thumb_tip - wrist
    
    # 벡터의 내적을 사용하여 손바닥의 방향 판단
    palm_normal_vector = np.cross(wrist_to_index, wrist_to_thumb)
    
    # 벡터의 z값을 확인 (손바닥이 카메라를 향할 때 z값이 양수임)
    return palm_normal_vector[2] > 0

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # MediaPipe로 손 랜드마크 검출
    result = hands.process(rgb_frame)

    hand_detected = False

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            hand_detected = True

            # 손바닥이 카메라를 향하고 있는지 판단
            if is_palm_facing_camera(hand_landmarks):
                color = (0, 255, 0)  # 초록색 (정상)
                status = "NORMAL"
            else:
                color = (0, 0, 255)  # 빨간색 (비정상)
                status = "DANGER"

            # 손 랜드마크를 이미지에 그림
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))

            # 바운딩 박스 그리기
            h, w, c = frame.shape
            x_min = min([landmark.x for landmark in hand_landmarks.landmark]) * w
            y_min = min([landmark.y for landmark in hand_landmarks.landmark]) * h
            x_max = max([landmark.x for landmark in hand_landmarks.landmark]) * w
            y_max = max([landmark.y for landmark in hand_landmarks.landmark]) * h
            cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color, 2)

            # 상태 텍스트 표시
            cv2.putText(frame, status, (int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    if not hand_detected:
        # 손이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\project\deep\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [1]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",  # 네트워크 아키텍처 파일
    "res10_300x300_ssd_iter_140000.caffemodel"  # 훈련된 모델 파일
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def is_face_facing_camera(rotation_vector):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    # 회전 벡터를 회전 행렬로 변환
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    
    # 회전 행렬을 오일러 각도로 변환
    pose_mat = np.hstack((rotation_matrix, np.array([[0], [0], [0]])))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_mat)
    
    pitch, yaw, roll = [math.radians(_) for _ in euler_angles]

    # 얼굴이 정면을 향하고 있는지 확인 (yaw와 pitch가 일정 범위 내에 있는지)
    return abs(yaw) < 10 and abs(pitch) < 10

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출 (단순히 중심에 있는 점을 사용하는 예시)
            image_points = np.array([
                ((x_min + x_max) / 2, (y_min + y_max) / 2),     # 중심
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if is_face_facing_camera(rotation_vector):
                color = (0, 255, 0)  # 초록색 (정상)
                status = "NORMAL"
            else:
                color = (0, 0, 255)  # 빨간색 (비정상)
                status = "DANGER"

            # 얼굴 주변에 bounding box 그리기
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
            cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            # 얼굴 방향 정보 표시
            cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    if not face_detected:
        # 얼굴이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\solvepnp.cpp:840: error: (-215:Assertion failed) ( (npoints >= 4) || (npoints == 3 && flags == SOLVEPNP_ITERATIVE && useExtrinsicGuess) || (npoints >= 3 && flags == SOLVEPNP_SQPNP) ) && npoints == std::max(ipoints.checkVector(2, CV_32F), ipoints.checkVector(2, CV_64F)) in function 'cv::solvePnPGeneric'


In [4]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",  # 네트워크 아키텍처 파일
    "res10_300x300_ssd_iter_140000.caffemodel"  # 훈련된 모델 파일
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def get_euler_angles(rotation_vector):
    """회전 벡터를 오일러 각도로 변환"""
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = np.hstack((rotation_matrix, np.array([[0], [0], [0]])))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_mat)
    return np.radians(euler_angles)

def is_face_facing_camera(rotation_vector):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    euler_angles = get_euler_angles(rotation_vector)
    pitch, yaw, roll = euler_angles[0], euler_angles[1], euler_angles[2]

    # 얼굴이 정면을 향하고 있는지 확인 (yaw와 pitch가 일정 범위 내에 있는지)
    return abs(yaw) < 10 and abs(pitch) < 10, pitch, yaw, roll

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출 (랜드마크 기반으로 업데이트 필요)
            image_points = np.array([
                (x_min, y_min),    # 코끝
                (x_min, y_max),    # 턱
                (x_min + (x_max - x_min) * 0.2, y_min + (y_max - y_min) * 0.2),  # 왼쪽 눈
                (x_min + (x_max - x_min) * 0.8, y_min + (y_max - y_min) * 0.2),  # 오른쪽 눈
                (x_min + (x_max - x_min) * 0.2, y_max - (y_max - y_min) * 0.2),  # 왼쪽 입
                (x_min + (x_max - x_min) * 0.8, y_max - (y_max - y_min) * 0.2)   # 오른쪽 입
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if success:
                face_facing_camera, pitch, yaw, roll = is_face_facing_camera(rotation_vector)
                if face_facing_camera:
                    color = (0, 255, 0)  # 초록색 (정상)
                    status = "NORMAL"
                else:
                    color = (0, 0, 255)  # 빨간색 (비정상)
                    status = "DANGER"

                # 얼굴 주변에 bounding box 그리기
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # 얼굴 방향 정보 표시
                cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    if not face_detected:
        # 얼굴이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\359622340.py:97: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\359622340.py:98: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\359622340.py:99: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you ext

In [5]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",  # 네트워크 아키텍처 파일
    "res10_300x300_ssd_iter_140000.caffemodel"  # 훈련된 모델 파일
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def get_euler_angles(rotation_vector):
    """회전 벡터를 오일러 각도로 변환"""
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = np.hstack((rotation_matrix, np.array([[0], [0], [0]])))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_mat)
    return np.radians(euler_angles)

def is_face_facing_camera(rotation_vector):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    euler_angles = get_euler_angles(rotation_vector)
    pitch, yaw, roll = euler_angles[0], euler_angles[1], euler_angles[2]

    # 얼굴이 정면을 향하고 있는지 확인 (yaw와 pitch가 일정 범위 내에 있는지)
    return abs(yaw) < 10 and abs(pitch) < 10, pitch, yaw, roll

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출 (랜드마크 기반으로 업데이트 필요)
            image_points = np.array([
                (x_min + (x_max - x_min) * 0.5, y_min + (y_max - y_min) * 0.5),  # 코끝
                (x_min, y_max),  # 턱
                (x_min + (x_max - x_min) * 0.2, y_min + (y_max - y_min) * 0.2),  # 왼쪽 눈
                (x_min + (x_max - x_min) * 0.8, y_min + (y_max - y_min) * 0.2),  # 오른쪽 눈
                (x_min + (x_max - x_min) * 0.2, y_max - (y_max - y_min) * 0.2),  # 왼쪽 입
                (x_min + (x_max - x_min) * 0.8, y_max - (y_max - y_min) * 0.2)   # 오른쪽 입
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if success:
                face_facing_camera, pitch, yaw, roll = is_face_facing_camera(rotation_vector)
                
                # 얼굴이 정면을 향할 때와 약간 틀릴 때 조건
                if face_facing_camera:
                    color = (0, 255, 0)  # 초록색 (정상)
                    status = "NORMAL"
                else:
                    color = (0, 0, 255)  # 빨간색 (비정상)
                    status = "DANGER"

                # 얼굴 주변에 bounding box 그리기
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # 얼굴 방향 정보 표시
                cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    if not face_detected:
        # 얼굴이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\913725031.py:99: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\913725031.py:100: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\913725031.py:101: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you e

In [6]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",  # 네트워크 아키텍처 파일
    "res10_300x300_ssd_iter_140000.caffemodel"  # 훈련된 모델 파일
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def get_euler_angles(rotation_vector):
    """회전 벡터를 오일러 각도로 변환"""
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = np.hstack((rotation_matrix, np.array([[0], [0], [0]])))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_mat)
    return np.radians(euler_angles)

def is_face_facing_camera(rotation_vector):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    euler_angles = get_euler_angles(rotation_vector)
    pitch, yaw, roll = euler_angles[0], euler_angles[1], euler_angles[2]

    # 각도 범위를 더 엄격하게 조정
    yaw_threshold = np.radians(5)  # 5도
    pitch_threshold = np.radians(5)  # 5도
    roll_threshold = np.radians(5)  # 5도

    is_facing = (abs(yaw) < yaw_threshold and 
                 abs(pitch) < pitch_threshold and 
                 abs(roll) < roll_threshold)

    return is_facing, pitch, yaw, roll

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출 (랜드마크 기반으로 업데이트 필요)
            image_points = np.array([
                (x_min + (x_max - x_min) * 0.5, y_min + (y_max - y_min) * 0.5),  # 코끝
                (x_min, y_max),  # 턱
                (x_min + (x_max - x_min) * 0.2, y_min + (y_max - y_min) * 0.2),  # 왼쪽 눈
                (x_min + (x_max - x_min) * 0.8, y_min + (y_max - y_min) * 0.2),  # 오른쪽 눈
                (x_min + (x_max - x_min) * 0.2, y_max - (y_max - y_min) * 0.2),  # 왼쪽 입
                (x_min + (x_max - x_min) * 0.8, y_max - (y_max - y_min) * 0.2)   # 오른쪽 입
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if success:
                face_facing_camera, pitch, yaw, roll = is_face_facing_camera(rotation_vector)
                
                # 상태 판단 로직 수정
                if face_facing_camera:
                    color = (0, 255, 0)  # 초록색 (정상)
                    status = "NORMAL"
                else:
                    color = (0, 0, 255)  # 빨간색 (비정상)
                    status = "DANGER"

                # 얼굴 주변에 bounding box 그리기
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # 얼굴 방향 정보 표시
                cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    if not face_detected:
        # 얼굴이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\209947352.py:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\209947352.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\209947352.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you 

In [11]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",  # 네트워크 아키텍처 파일
    "res10_300x300_ssd_iter_140000.caffemodel"  # 훈련된 모델 파일
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def get_euler_angles(rotation_vector):
    """회전 벡터를 오일러 각도로 변환"""
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    pose_mat = np.hstack((rotation_matrix, np.array([[0], [0], [0]])))
    _, _, _, _, _, _, euler_angles = cv2.decomposeProjectionMatrix(pose_mat)
    return np.radians(euler_angles)

def is_face_facing_camera(rotation_vector):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    euler_angles = get_euler_angles(rotation_vector)
    pitch, yaw, roll = euler_angles[0], euler_angles[1], euler_angles[2]

    # 각도 범위를 더 엄격하게 조정
    yaw_threshold = np.radians(5)  # 5도
    pitch_threshold = np.radians(5)  # 5도
    roll_threshold = np.radians(5)  # 5도

    is_facing = (abs(yaw) < yaw_threshold and 
                 abs(pitch) < pitch_threshold and 
                 abs(roll) < roll_threshold)

    return is_facing, pitch, yaw, roll

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출 (랜드마크 기반으로 업데이트 필요)
            image_points = np.array([
                (x_min + (x_max - x_min) * 0.5, y_min + (y_max - y_min) * 0.5),  # 코끝
                (x_min, y_max),  # 턱
                (x_min + (x_max - x_min) * 0.2, y_min + (y_max - y_min) * 0.2),  # 왼쪽 눈
                (x_min + (x_max - x_min) * 0.8, y_min + (y_max - y_min) * 0.2),  # 오른쪽 눈
                (x_min + (x_max - x_min) * 0.2, y_max - (y_max - y_min) * 0.2),  # 왼쪽 입
                (x_min + (x_max - x_min) * 0.8, y_max - (y_max - y_min) * 0.2)   # 오른쪽 입
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if success:
                face_facing_camera, pitch, yaw, roll = is_face_facing_camera(rotation_vector)
                
                # 상태 판단 로직 수정
                if face_facing_camera:
                    color = (0, 255, 0)  # 초록색 (정상)
                    status = "NORMAL"
                else:
                    color = (0, 0, 255)  # 빨간색 (비정상)
                    status = "DANGER"

                # 얼굴 주변에 bounding box 그리기
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # 얼굴 방향 정보 표시
                cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    if not face_detected:
        # 얼굴이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\209947352.py:107: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\209947352.py:108: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\209947352.py:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you 

In [12]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",
    "res10_300x300_ssd_iter_140000.caffemodel"
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def get_euler_angles(rotation_vector):
    """회전 벡터를 오일러 각도로 변환"""
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    euler_angles = cv2.decomposeProjectionMatrix(np.hstack((rotation_matrix, np.zeros((3, 1)))))[6]
    return [math.radians(angle) for angle in euler_angles]

def is_face_facing_camera(euler_angles):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    pitch, yaw, roll = euler_angles

    # 각도 범위 조정 (단위: 라디안)
    yaw_threshold = math.radians(20)
    pitch_threshold = math.radians(20)
    roll_threshold = math.radians(20)

    is_facing = (abs(yaw) < yaw_threshold and 
                 abs(pitch) < pitch_threshold and 
                 abs(roll) < roll_threshold)

    return is_facing, pitch, yaw, roll

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출
            image_points = np.array([
                ((x_min + x_max) // 2, (y_min + y_max) // 2),  # 코끝
                ((x_min + x_max) // 2, y_max),                 # 턱
                (x_min, (y_min + y_max) // 2),                 # 왼쪽 눈
                (x_max, (y_min + y_max) // 2),                 # 오른쪽 눈
                (x_min, y_max - (y_max - y_min) // 3),         # 왼쪽 입
                (x_max, y_max - (y_max - y_min) // 3)          # 오른쪽 입
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if success:
                euler_angles = get_euler_angles(rotation_vector)
                face_facing_camera, pitch, yaw, roll = is_face_facing_camera(euler_angles)
                
                if face_facing_camera:
                    color = (0, 255, 0)  # 초록색 (정상)
                    status = "NORMAL"
                else:
                    color = (0, 0, 255)  # 빨간색 (비정상)
                    status = "DANGER"

                # 얼굴 주변에 bounding box 그리기
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # 얼굴 방향 정보 표시
                cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.HOTERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2460\3176102665.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [math.radians(angle) for angle in euler_angles]


AttributeError: module 'cv2' has no attribute 'HOTERSHEY_SIMPLEX'

In [3]:
import cv2
import numpy as np
import math

# 얼굴 검출기 로드 (OpenCV DNN 모듈)
net = cv2.dnn.readNetFromCaffe(
    "deploy.prototxt.txt",
    "res10_300x300_ssd_iter_140000.caffemodel"
)

# 3D 모델 포인트
model_points = np.array([
    (0.0, 0.0, 0.0),             # 코끝
    (0.0, -330.0, -65.0),        # 턱
    (-225.0, 170.0, -135.0),     # 왼쪽 눈 왼쪽 구석
    (225.0, 170.0, -135.0),      # 오른쪽 눈 오른쪽 구석
    (-150.0, -150.0, -125.0),    # 왼쪽 입 구석
    (150.0, -150.0, -125.0)      # 오른쪽 입 구석
])

def get_euler_angles(rotation_vector):
    """회전 벡터를 오일러 각도로 변환"""
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    euler_angles = cv2.decomposeProjectionMatrix(np.hstack((rotation_matrix, np.zeros((3, 1)))))[6]
    return [math.radians(angle) for angle in euler_angles]

def is_face_facing_camera(euler_angles):
    """얼굴이 카메라를 향하고 있는지 확인하는 함수"""
    pitch, yaw, roll = euler_angles

    # 각도 범위 조정 (단위: 라디안)
    yaw_threshold = math.radians(20)
    pitch_threshold = math.radians(20)
    roll_threshold = math.radians(20)

    is_facing = (abs(yaw) < yaw_threshold and 
                 abs(pitch) < pitch_threshold and 
                 abs(roll) < roll_threshold)

    return is_facing, pitch, yaw, roll

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width = frame.shape[:2]

    # 카메라 행렬 정의
    focal_length = width
    center = (width / 2, height / 2)
    camera_matrix = np.array(
        [[focal_length, 0, center[0]],
         [0, focal_length, center[1]],
         [0, 0, 1]], dtype="double"
    )

    # 이미지 전처리
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()

    face_detected = False

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            face_detected = True
            box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x_min, y_min, x_max, y_max) = box.astype("int")

            # 2D 이미지 포인트 추출
            image_points = np.array([
                ((x_min + x_max) // 2, (y_min + y_max) // 2),  # 코끝
                ((x_min + x_max) // 2, y_max),                 # 턱
                (x_min, (y_min + y_max) // 2),                 # 왼쪽 눈
                (x_max, (y_min + y_max) // 2),                 # 오른쪽 눈
                (x_min, y_max - (y_max - y_min) // 3),         # 왼쪽 입
                (x_max, y_max - (y_max - y_min) // 3)          # 오른쪽 입
            ], dtype="double")

            # SolvePnP를 사용하여 회전 벡터와 변환 벡터 계산
            dist_coeffs = np.zeros((4, 1))  # 카메라 왜곡 계수, 여기서는 0으로 가정
            (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE)

            if success:
                euler_angles = get_euler_angles(rotation_vector)
                face_facing_camera, pitch, yaw, roll = is_face_facing_camera(euler_angles)
                
                if face_facing_camera:
                    color = (0, 255, 0)  # 초록색 (정상)
                    status = "NORMAL"
                else:
                    color = (0, 0, 255)  # 빨간색 (비정상)
                    status = "DANGER"

                # 얼굴 주변에 bounding box 그리기
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                cv2.putText(frame, status, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

                # 얼굴 방향 정보 표시
                cv2.putText(frame, f"Pitch: {int(math.degrees(pitch))}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Yaw: {int(math.degrees(yaw))}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Roll: {int(math.degrees(roll))}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\Admin\AppData\Local\Temp\ipykernel_48260\2566325854.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [math.radians(angle) for angle in euler_angles]
